In [1]:
# Import Library
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from dash.dependencies import Input, Output, State
from io import BytesIO
import requests
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
import plotly.express as px
import dash_table as dt
#import dash_colorscales

In [2]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__,external_stylesheets=[dbc.themes.DARKLY])

In [3]:
rfc_loc = 'https://github.com/pranaymallipudi/academics/blob/main/model.pkl?raw=true'
Scaler_loc = 'https://github.com/pranaymallipudi/academics/blob/main/scaler.pkl?raw=true'

mfile_rfc = BytesIO(requests.get(rfc_loc).content)
rfc = pickle.load(mfile_rfc)

mfile_sca = BytesIO(requests.get(Scaler_loc).content)
Scaler = pickle.load(mfile_sca)


attrition = pd.read_csv('https://raw.githubusercontent.com/pranaymallipudi/academics/main/Final%20dataset%20Attrition.csv', error_bad_lines=False)
attrition2 = attrition[:]
bins = [0, 5000, 10000, 15000, 20000]
attrition2['MonthlyIncomeBinned'] = pd.cut(attrition2['MonthlyIncome'],bins)
df = pd.read_csv('https://raw.githubusercontent.com/pranaymallipudi/academics/main/feature_imp.csv')
df.set_index('feature', inplace=True)
#fig = px.bar(df, y = df.index,x = df['importance'],orientation='h',title='Feature Importance according to Random Forest')
#fig.show()

/Users/shailendra_narwaria/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/shailendra_narwaria/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Users/shailendra_narwaria/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 1.0.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
<ipython-input-3-b5b6e047eace>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [4]:
app.layout = html.Div([
    #html.H1('TURNOVER DASHBOARD'),
    html.H1('TURNOVER DASHBOARD', style={'fontSize':40,'width':'100%', 'height': '100%','margin':25, 'textAlign': 'center','color': '#89AEDB'}),
    dcc.Tabs(id="tabs-example-graph", value='tab-1-example-graph', children=[
        
        dcc.Tab(label='Attrition',value='tab-1-example-graph',
                style={ 'textAlign': 'center','color': '#89AEDB','background': '#272E3E','fontSize': '50'},
                selected_style = {'backgroundColor': '#272E3E',
    'color': '#89AEDB','borderTop': '3px solid #d6d6d6',}),
        dcc.Tab(label='Select Tab to Predict Attrition', value='tab-2-example-graph',
                style={'textAlign': 'center','color': '#89AEDB','background': '#272E3E','fontSize': '50'},
                selected_style = {'backgroundColor': '#272E3E',
    'color': '#89AEDB','borderTop': '3px solid #d6d6d6'}),
       
    ],
        colors={
        "border": "#272E3E",
        "primary": "272E3E",
        "background": "#272E3E"
    },
            ),
    html.Div(id='tabs-content-example-graph'),
    
    
])


In [5]:
@app.callback(Output('tabs-content-example-graph', 'children'),
              Input('tabs-example-graph', 'value'))
def render_content(tab):
    if tab == 'tab-1-example-graph':
            return html.Div(children = [
        # Title
        #html.H1('IBM HR Attrition Data', style={'textAlign':'center','color': 'dark-grey', 'fontSize': '50'}),
        # Source Link
        html.P(['Source: ',
                html.A('https://www.kaggle.com/singhnproud77/hr-attrition-dataset',
                href='https://www.kaggle.com/singhnproud77/hr-attrition-dataset'),
        ],style = {'textAlign':'center','color': '#89AEDB'}),
        # Empty Line
        html.P(''),
        # Filter Headers
        dbc.Row([
            dbc.Col(html.P('Department',
                           style = {
                            'textAlign':'center',
                            'fontSize':25,'color': '#89AEDB','background': '#272E3E'}),width = 4),
            dbc.Col(html.P('Job Role',
                           style = {
                            'textAlign':'center',
                            'fontSize':25,'color': '#89AEDB','background': '#272E3E'}),width = 4),
            dbc.Col(html.P('Age',
                           style = {
                            'textAlign':'center',
                            'fontSize':25,'color': '#89AEDB','background': '#272E3E'}),width = 4),
        ]),
        # Filters
        dbc.Row([
            dbc.Col(dcc.Dropdown(
                id='dept-dropdown',
                options=[
                    {'label': i, 'value': i} for i in list(set(attrition['Department']))+['All Departments']],
                value='All Departments', style={'color': '#89AEDB','background': '#89AEDB' } 
                ),width = 4,  ),
            dbc.Col(dcc.Dropdown(
                id='job-role',
                options=[
                    {'label': i, 'value': i} for i in list(set(attrition['JobRole']))+['All Jobs']],
                value='All Jobs', style={'color': '#89AEDB','background': '#89AEDB' } 
                ),width = 4, ),
            dbc.Col(dcc.RangeSlider(
                id='age-slider',
                min=min(attrition['Age']),
                max=max(attrition['Age']),
                step=1,
                value=[25,30],
                tooltip={"placement": "bottom", "always_visible": True},
                ),width = 4)
        ],),  
        # KPI Headers
        dbc.Row([
            dbc.Col(html.H6(children = 'Attrition Rate',
                            style = {
                                'textAlign':'center',
                                'fontSize':25,'color': '#89AEDB',
                                }
                            ),width = 4,),
            dbc.Col(html.H6(children = 'Average Tenure',
                            style = {
                                'textAlign':'center',
                                'fontSize':25,'color': '#89AEDB',
                                }
                            ),width = 4,),
            dbc.Col(html.H6(children = 'Number of Employees',
                        style = {
                            'textAlign':'center',
                            'fontSize':25,'color': '#89AEDB',
                            }),width = 4,),
            ]),
        # KPI Metrics
        dbc.Row([
            dbc.Col(html.P(id = 'attrition_rate',
                           #children = f"{round(len(attrition[attrition['Attrition']=='Yes'])/len(attrition)*100,2)}%",
                                                style = {
                                                    'textAlign':'center','color': '#89AEDB',
                                                    'fontSize':30,'background': '#272E3E',
                                                }),width = 4,),
            dbc.Col(html.P(id = 'average_time_at_company',
                        #children = f"{round(attrition['YearsAtCompany'].mean(),2)} Years",
                        style = {
                            'textAlign':'center','color': '#89AEDB',
                            'fontSize':30,'background': '#272E3E',
               }),width = 4,),
            dbc.Col(html.P(id  = 'number_of_employees',
                        #children = f"{attrition.shape[0]}",
                        style = {
                            'textAlign':'center','color': '#89AEDB',
                            'fontSize':30,'background': '#272E3E',
                        }),width = 4,),
            ]),
        # Graphs 1 & 2
        dbc.Row([
            dbc.Col(dcc.Graph(id ='graph_1'),width = 6),
            dbc.Col(dcc.Graph(id ='graph_2'),width = 6),
        ]),
        # Attrition Graph
        dbc.Row([
            dbc.Col(dcc.Graph(id = 'attrition-graph'),width = 6 ),
            dbc.Col(dcc.Graph(id ='graph_3'),width = 6),
        ]),

        # Data Table
        dt.DataTable(
            id='data-table',
            columns=[{"name": i, "id": i} for i in ['MonthlyIncome','DistanceFromHome','TotalWorkingYears','PercentSalaryHike','OverTime','YearsAtCompany','NumCompaniesWorked','YearsWithCurrManager','YearsSinceLastPromotion']],
        style_data={
        'color': '#83D2B7',
        'backgroundColor': '#111111'
    },
    style_header={
        'backgroundColor': '#272E3E',
        'color': '#89AEDB',
        'fontWeight': 'bold'
    }),
        # Parallel Categories Plot
#         dcc.Graph(
#             id='parralel categories',
#             figure = px.parallel_categories(attrition, ['Attrition','Gender', 'Department'],
#             labels={'Attrition':'Attrition', 'Gender':'Sex', 'Department':'Department'},)
#                 ),
        # dcc.Graph(id='graph-with-slider',
        #           figure = px.box(attrition, x= 'Attrition',y="MonthlyIncome"),
        #           ),
    ])
    elif tab == 'tab-2-example-graph':
            return html.Div(children=[ #children is to make child HTML tags
        html.H1('Chances of attrition',style = {
                                                    'textAlign':'center','color': '#89AEDB',
                                                    'fontSize':40,'background': '#272E3E',
                                                }), #H1 is for website header
        html.Div([
            html.Div([
                html.Div(id='output_gender', style={'font-weight': 'bold','color': '#89AEDB'}),
                dcc.Dropdown(
                    id='gender',
                    options=[
                        {'label': 'Female', 'value': 'Female'},
                        {'label': 'Male', 'value': 'Male'}
                    ],value='Female' ,style={'color': '#89AEDB','background': '#89AEDB', }
                ),  
                html.Div(id='output_age', style={'font-weight': 'bold','color': '#89AEDB'}),
                dcc.Slider(
                    id='age',
                    min=18,
                    max=60,
                    step=1,
                    marks={18: '18th', 60: '60 th'},
                    value=30
                ),

                html.Div(id='output_JobRole', style={'font-weight': 'bold','color': '#89AEDB'}),
                dcc.Dropdown(
                    id='JobRole',
                    options=[
                        {'label': 'Healthcare Representative', 'value': 'Healthcare Representative'},
                        {'label': 'Human Resources', 'value': 'Human Resources'},
                        {'label': 'Laboratory Technician', 'value': 'Laboratory Technician'},
                        {'label': 'Manager', 'value': 'Manager'},
                        {'label': 'Manufacturing Director', 'value': 'Manufacturing Director'},
                        {'label': 'Research Director', 'value': 'Research Director'},
                        {'label': 'Research Scientist', 'value': 'Research Scientist'},
                        {'label': 'Sales Executive', 'value': 'Sales Executive'},
                        {'label': 'Sales Representative', 'value': 'Sales Representative'},
                    ],value = 'Manager', style={'color': '#89AEDB','background': '#89AEDB', }
                ),

                html.Div(id='output_holsat', style={'font-weight': 'bold', 'color': '#89AEDB'}),
                dcc.Slider(
                    id='holsat',
                    min=0,
                    max=20,
                    step=1,
                    marks={0: '18th', 20: '60 th'},
                    value=10
                ),

                html.Div(id='output_YearsInMyCompany', style={'font-weight': 'bold', 'color': '#89AEDB'}),
                dcc.Slider(
                    id='YearsInMyCompany',
                    min=0,
                    max=40,
                    step=1,
                    marks={0:'0', 40: '40'},
                    value=20
                ),


                html.Div(id='output_CurrManager', style={'font-weight': 'bold', 'color': '#89AEDB'}),
                dcc.Slider(
                    id='CurrManager',
                    min=0,
                    max=20,
                    step=1,
                    marks={0:'0', 20:'20'},
                    value=1
                ),
#                 html.Div(id='feature_imp', style={'font-weight': 'bold'}),
#                 dcc.Graph(figure=fig)

    #################   
            ], className="six columns",style={'width': '49%', 'display': 'inline-block','height': '100vh'}),
            html.Div([
                  html.Div(id='output_LastPromotion', style={'font-weight': 'bold', 'color': '#89AEDB'}),
                dcc.Slider(
                    id='LastPromotion',
                    min=0,
                    max=15,
                    step=1,
                    marks={0:'0', 15:'15'},
                    value=3
                ),

                html.Div(id='output_totalworking', style={'font-weight': 'bold', 'color': '#89AEDB'}),
                dcc.Slider(
                    id='TotalWorking',
                    min=0,
                    max=40,
                    step=1,
                    marks={0:'0', 40: '40'},
                    value=20
                ),

                html.Div(id='output_numcompanies', style={'font-weight': 'bold', 'color': '#89AEDB'}),
                dcc.Slider(
                    id='NumCompanies',
                    min=0,
                    max=10,
                    step=1,
                    marks={0:'0', 10:'10'},
                    value=3
                ),

                html.Div(id='output_distance', style={'font-weight': 'bold', 'color': '#89AEDB'}),
                dcc.Slider(
                    id='distance',
                    min=0,
                    max=30,
                    step=1,
                    marks={0: '0 km', 30: '30 km'},
                    value=10
                ),
                html.Div(id='output_ot', style={'font-weight': 'bold', 'color': '#89AEDB'}),
                dcc.Dropdown(
                    id='ot',
                    options=[
                        {'label': 'Yes', 'value': 'Yes'},
                        {'label': 'No', 'value': 'No'}
                    ],value='No',style={'color': '#89AEDB','background': '#89AEDB', }
                ),  

                html.Div(id='output_PercentSalaryHike', style={'font-weight': 'bold', 'color': '#89AEDB'}),
                dcc.Slider(
                    id='PercentSalaryHike',
                    min=10,
                    max=25,
                    step=1,
                    marks={10:'10',25:'25'},
                    value=15
                ),

                html.Div(id='out-all-types', style={'font-weight': 'bold', 'color': '#89AEDB'}),
                dcc.Input(
                    id="input_{}".format("number"),
                    type="number",
                    placeholder="Monthly Salary {}".format("number"),
                    value = 5000),

                html.H3(id='pred_result', style={'font-weight': 'bold', 'color': '#89AEDB'})
            ], className = "six columns", style={'width': '50%', 'float': 'right', 'display': 'inline-block','height' : '80vh'}),
        ], className = "row",style ={'display': 'inline-block','width': '100%','height':'100%'})
    ################# 
    ], style={'padding': '15px 10px','height':'100vh'})

        
@app.callback(
Output('job-role', 'children'),
Input('dept-dropdown', 'value'),
)
def job_dropdown(dept):
    if dept == 'All Departments':
        jobs = list(set(attrition['JobRole']))
    else:
        jobs = list(set(attrition[attrition['Department'] == dept]['JobRole']))
    return jobs

@app.callback(
    Output('attrition_rate', 'children'),
    Input('dept-dropdown', 'value'),
    Input('job-role','value'),
    Input('age-slider','value'),
)
def select_dept(dept,job,age):
    if dept == 'All Departments':
        if job == 'All Jobs':
            attrition_rate = f"{round(len(attrition[(attrition['Attrition']=='Yes') & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])])/len(attrition[(attrition['Age']>=age[0]) & (attrition['Age']<=age[1])])*100,2)}%"
        else:
            attrition_rate = f"{round(len(attrition[(attrition['Attrition']=='Yes') & (attrition['JobRole']==job) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])])/len(attrition[(attrition['JobRole']==job) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])])*100,2)}%"
    else:
        if job == 'All Jobs':
            attrition_rate = f"{round(len(attrition[(attrition['Attrition']=='Yes') & (attrition['Department']==dept) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])])/len(attrition[(attrition['Department']==dept) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])])*100,2)}%"
        else:
            attrition_rate = f"{round(len(attrition[(attrition['Attrition']=='Yes') & (attrition['Department']==dept) & (attrition['JobRole']==job) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])])/len(attrition[(attrition['Department']==dept) & (attrition['JobRole']==job) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])])*100,2)}%"
    return attrition_rate

@app.callback(
    Output('average_time_at_company', 'children'),
    Input('dept-dropdown', 'value'),
    Input('job-role','value'),
    Input('age-slider','value'),
)
def select_job(dept,job,age):
    if dept == 'All Departments':
        if job == 'All Jobs':
            time_at_org = round(attrition[(attrition['Age']>=age[0]) & (attrition['Age']<=age[1])]['YearsAtCompany'].mean(),2)
        else:
            time_at_org = round(attrition[(attrition['JobRole']==job) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])]['YearsAtCompany'].mean(),2)
    else:
        if job == 'All Jobs':
            time_at_org = round(attrition[(attrition['Department']==dept) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])]['YearsAtCompany'].mean(),2)
        else:
            time_at_org = round(attrition[(attrition['Department']==dept) & (attrition['JobRole']==job) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])]['YearsAtCompany'].mean(),2)
    return time_at_org

@app.callback(
    Output('number_of_employees', 'children'),
    Input('dept-dropdown', 'value'),
    Input('job-role','value'),
    Input('age-slider','value'),
)
def number_employees(dept,job,age):
    if dept == 'All Departments':
        if job == 'All Jobs':
            total = attrition[(attrition['Age']>=age[0]) & (attrition['Age']<=age[1])].shape[0]
        else:
            total = attrition[(attrition['JobRole']==job) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])].shape[0]
    else:
        if job == 'All Jobs':
            total = attrition[(attrition['Department']==dept) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])].shape[0]
        else: 
            total = attrition[(attrition['Department']==dept) & (attrition['JobRole']==job) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])].shape[0]
    return total

@app.callback(Output('graph_1', 'figure'),
            Input('dept-dropdown', 'value'),
            Input('job-role','value'),
            Input('age-slider','value'),
)
def final_fig(dept,job,age):
    attrition['EmployeeCount'] = 1
    if dept == 'All Departments':
        if job == 'All Jobs':
            grouped_df = attrition[(attrition['Age']>=age[0]) & (attrition['Age']<=age[1])].groupby(by = ['Gender','Attrition','Department'],as_index = False).agg(
                {'EmployeeCount':'count'})
        else:
            grouped_df = attrition[(attrition['JobRole']==job) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])].groupby(by = ['Gender','Attrition','Department'],as_index = False).agg(
                {'EmployeeCount':'count'}
        )
    else:
        if job == 'All Jobs':
            grouped_df = attrition[(attrition['Department']==dept) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])].groupby(by = ['Gender','Attrition','Department'],as_index = False).agg(
                {'EmployeeCount':'count'}
        )
        else:
            grouped_df = attrition[(attrition['Department']==dept) & (attrition['JobRole']==job) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])].groupby(by = ['Gender','Attrition','Department'],as_index = False).agg(
                {'EmployeeCount':'count'}
        )
    fig = px.bar(grouped_df, x="Gender", y="EmployeeCount",color = 'Attrition',color_discrete_map={'No':"#898989",'Yes': "#83D2B7"},template = 'plotly_dark',
            barmode = 'stack')
    return fig

@app.callback(Output('graph_2', 'figure'),
            Input('dept-dropdown', 'value'),
            Input('job-role','value'),
            Input('age-slider','value'),)
def scatter_fig(dept,job,age):
    if dept == 'All Departments':
        if job == 'All Jobs':
            fig = px.scatter(attrition[(attrition['Age']>=age[0]) & (attrition['Age']<=age[1])], x="YearsAtCompany", y="MonthlyIncome",color = 'Attrition'
            ,template = 'plotly_dark',color_discrete_map={'No':"#898989",'Yes': "#83D2B7"}            )
        else:
            fig = px.scatter(attrition[(attrition['JobRole']==job) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])], x="YearsAtCompany", y="MonthlyIncome",color = 'Attrition'
            ,template = 'plotly_dark',color_discrete_map={'No':"#898989",'Yes': "#83D2B7"}           )
    else:
        if job == 'All Jobs':
            fig = px.scatter(attrition[(attrition['Department']==dept) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])], x="YearsAtCompany", y="MonthlyIncome",color = 'Attrition'
            ,template = 'plotly_dark',color_discrete_map={'No':"#898989",'Yes': "#83D2B7"}           )
        else:
            fig = px.scatter(attrition[(attrition['Department']==dept) & (attrition['JobRole']==job) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])], x="YearsAtCompany", y="MonthlyIncome",color = 'Attrition'
            ,template = 'plotly_dark',color_discrete_map={'No':"#898989",'Yes': "#83D2B7"}           )
    return fig

@app.callback(Output('attrition-graph', 'figure'),
            [Input('dept-dropdown', 'value'),
            Input('job-role','value'),
            Input('age-slider','value')])
def attrition_graph(dept,job,age):
   
    if dept == 'All Departments':
        if job == 'All Jobs':
            df = attrition2[(attrition2['Age']>=age[0]) & (attrition2['Age']<=age[1])]
            grouped_df = df.groupby(['MonthlyIncomeBinned','Attrition']).agg({'Age':'count'} ).rename(columns = {'Age':'Count'}).reset_index()
            grouped_df['MonthlyIncomeBinned'] = grouped_df['MonthlyIncomeBinned'].astype(str)
            fig = px.bar(grouped_df,x = 'MonthlyIncomeBinned',y = 'Count',color = 'Attrition', barmode = 'group',template = 'plotly_dark',color_discrete_map={'No':"#898989",'Yes': "#83D2B7"} )

            #fig = px.bar(df, x=df['Attrition'], y = df['MonthlyIncome'])
        else:
            df = attrition2[(attrition2['JobRole']==job) & (attrition2['Age']>=age[0]) & (attrition2['Age']<=age[1])]
            grouped_df = df.groupby(['MonthlyIncomeBinned','Attrition']).agg({'Age':'count'} ).rename(columns = {'Age':'Count'}).reset_index()
            grouped_df['MonthlyIncomeBinned'] = grouped_df['MonthlyIncomeBinned'].astype(str)
            fig = px.bar(grouped_df,x = 'MonthlyIncomeBinned',y = 'Count',color = 'Attrition', barmode = 'group',template = 'plotly_dark',color_discrete_map={'No':"#898989",'Yes': "#83D2B7"} )
            #fig = px.bar(df, x=df['Attrition'], y = df['MonthlyIncome'])
    else:
        if job == 'All Jobs':
            df = attrition2[(attrition2['Department']==dept) & (attrition2['Age']>=age[0]) & (attrition2['Age']<=age[1])]
            grouped_df = df.groupby(['MonthlyIncomeBinned','Attrition']).agg({'Age':'count'} ).rename(columns = {'Age':'Count'}).reset_index()
            grouped_df['MonthlyIncomeBinned'] = grouped_df['MonthlyIncomeBinned'].astype(str)
            fig = px.bar(grouped_df,x = 'MonthlyIncomeBinned',y = 'Count',color = 'Attrition', barmode = 'group',template = 'plotly_dark',color_discrete_map={'No':"#898989",'Yes': "#83D2B7"} )
            #fig = px.bar(df, x=df['Attrition'], y = df['MonthlyIncome'])
        else:
            df = attrition[(attrition2['Department']==dept) & (attrition2['JobRole']==job) & (attrition2['Age']>=age[0]) & (attrition2['Age']<=age[1])]
            grouped_df = df.groupby(['MonthlyIncomeBinned','Attrition']).agg({'Age':'count'} ).rename(columns = {'Age':'Count'}).reset_index()
            grouped_df['MonthlyIncomeBinned'] = grouped_df['MonthlyIncomeBinned'].astype(str)
            fig = px.bar(grouped_df,x = 'MonthlyIncomeBinned',y = 'Count',color = 'Attrition', barmode = 'group',template = 'plotly_dark',color_discrete_map={'No':"#898989",'Yes': "#83D2B7"} )
            #fig = px.bar(df, x=df['Attrition'], y = df['MonthlyIncome'])
    return fig

@app.callback(Output('graph_3', 'figure'),
            Input('dept-dropdown', 'value'),
            Input('job-role','value'),
            Input('age-slider','value'),)
def status_leaving (dept,job,age):
    attrition['EmployeeCount'] = 1
    
    if dept == 'All Departments':
        if job == 'All Jobs':
            grouped_df = attrition[(attrition['Age']>=age[0]) & (attrition['Age']<=age[1]) & (attrition['Attrition']=='Yes')].groupby(by = ['Status_of_leaving','Attrition'],as_index = False).agg(
                {'EmployeeCount':'count'})
        else:
            grouped_df = attrition[(attrition['JobRole']==job) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])& (attrition['Attrition']=='Yes')].groupby(by = ['Status_of_leaving','Attrition'],as_index = False).agg(
                {'EmployeeCount':'count'}
        )
    else:
        if job == 'All Jobs':
            grouped_df = attrition[(attrition['Department']==dept) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])& (attrition['Attrition']=='Yes')].groupby(by = ['Status_of_leaving','Attrition'],as_index = False).agg(
                {'EmployeeCount':'count'}
        )
        else:
            grouped_df = attrition[(attrition['Department']==dept) & (attrition['JobRole']==job) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])& (attrition['Attrition']=='Yes')].groupby(by = ['Status_of_leaving','Attrition'],as_index = False).agg(
                {'EmployeeCount':'count'}
        )
    fig = px.bar(grouped_df, x="EmployeeCount", y="Status_of_leaving",color = 'Attrition',
            barmode = 'stack',orientation='h',template = 'plotly_dark',color_discrete_map={'No':"#898989",'Yes': "#83D2B7"}  )
    return fig

@app.callback(Output('data-table', 'data'),
            [Input('dept-dropdown', 'value'),
            Input('job-role','value'),
            Input('age-slider','value')])
def data_table(dept,job,age):
    if dept == 'All Departments':
        if job == 'All Jobs':
            filtered_df = attrition[(attrition['Age']>=age[0]) & (attrition['Age']<=age[1])]
        else:
            filtered_df = attrition[(attrition['JobRole']==job) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])]
    else:
        if job == 'All Jobs':
            filtered_df = attrition[(attrition['Department']==dept) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])]
        else:
            filtered_df = attrition[(attrition['Department']==dept) & (attrition['JobRole']==job) & (attrition['Age']>=age[0]) & (attrition['Age']<=age[1])]
    data=filtered_df.to_dict('records')
    return data

##################
@app.callback(
    Output('output_gender', 'children'),
    [Input('gender', 'value')])
def update_output(value):
    return 'Gender: {}'.format(value)

@app.callback(
    Output('output_age', 'children'),
    [Input('age', 'value')])
def update_output(value):
    return 'Age: {} years'.format(value)
    
@app.callback(
    Output('output_JobRole', 'children'),
    [Input('JobRole', 'value')])
def update_output(value):
    return 'Job Role: {}'.format(value)  

@app.callback(
    Output('output_holsat', 'children'),
    [Input('holsat', 'value')])
def update_output(value):
    return 'Holistic Satisfaction: {} '.format(value)
    
@app.callback(
    Output('output_CurrManager', 'children'),
    [Input('CurrManager', 'value')])
def update_output(value):
    return 'Total Years with Current Manager: {} years'.format(value)

@app.callback(
    Output('output_YearsInMyCompany', 'children'),
    [Input('YearsInMyCompany', 'value')])
def update_output(value):
    return 'Total Years Employee at My Company: {} years'.format(value)


##########
@app.callback(
    Output('output_LastPromotion', 'children'),
    [Input('LastPromotion', 'value')])
def update_output(value):
    return 'Total Years Since Last Promotion: {} years'.format(value)
    
@app.callback(
    Output('output_totalworking', 'children'),
    [Input('TotalWorking', 'value')])
def update_output(value):
    return 'Total Years of Experience: {} years'.format(value)
    
@app.callback(
    Output('output_numcompanies', 'children'),
    [Input('NumCompanies', 'value')])
def update_output(value):
    return 'Total Number of Companies: {}'.format(value)
    

@app.callback(
    Output('output_distance', 'children'),
    [Input('distance', 'value')])
def update_output(value):
    return 'Distance From Home: {} km'.format(value)
    
@app.callback(
    Output('output_ot', 'children'),
    [Input('ot', 'value')])
def update_output(value):
    return 'Overtime: {}'.format(value)

@app.callback(
    Output('output_PercentSalaryHike', 'children'),
    [Input('PercentSalaryHike', 'value')])
def update_output(value):
    return 'Percent Salary Hike Last Year: {}'.format(value)

@app.callback(
    Output('out-all-types', 'children'),
    [Input("input_{}".format("number"), "value")])
def update_output(value):
    return 'Monthly Income: {} $'.format(value)
    
##################


@app.callback(
    Output('pred_result', 'children'),
    [Input('gender', 'value'),
     Input('age', 'value'),
    Input('JobRole', 'value'),
    Input('holsat', 'value'),
    Input('YearsInMyCompany', 'value'),
    Input('CurrManager', 'value'),
    Input('LastPromotion', 'value'),
    Input('TotalWorking', 'value'),
    Input('NumCompanies', 'value'),
    Input('distance', 'value'),
    Input('ot', 'value'),
    Input('PercentSalaryHike', 'value'),
    Input("input_{}".format("number"), 'value')])
  

def update_prediction(x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12, x13):
    #PREPROCESSING DATA INPUT
    #gender
    if (x1== 'Female'):
        x1=0
    elif (x1== 'Male'):
        x1=1
    #jobrole
    if (x3== 'Healthcare Representative'):
        x3=0
    elif (x3== 'Human Resources'):
        x3=1
    elif (x3== 'Laboratory Technician'):
        x3=2    
    elif (x3== 'Manager'):
        x3=3
    elif (x3== 'Manufacturing Director'):
        x3=4
    elif (x3== 'Research Director'):
        x3=5
    elif (x3== 'Research Scientist'):
        x3=6
    elif (x3== 'Sales Executive'):
        x3=7
    elif (x3== 'Sales Representative'):
        x3=8
    #overtime
    if (x11== 'Yes'):
        x11=1
    elif (x11== 'No'):
        x11=0
    
#     x2= float(x2)
#     x4= float(x4)
#     x5= float(x5)
#     x6= float(x6)
#     x7= float(x7)
#     x8= float(x8)
#     x9= float(x9)
#     x10= float(x10)
#     x12= float(x12)
#     x13= float(x13)
        
    
    # Create a NumPy array in the form of the original features
    input_X = np.array([x13, #Monthly income
                        x11, #overtime
                        x2, #Age
                        x10, #Distance From Home
                        x8, #Total working years
                        x12, #Percent salary hike
                        x5, #Years at company
                        x6, #Years with currrent manager
                        x9, #Number of companies worked
                        x7, #Years since last promotion
                        x4, #holistic satisfaction
                        x1, #Gender
                        x3, #Jobrole
                       ]).reshape(1,-1)
    #input_X = (input_X.astype(float))
    input_X = Scaler.transform(input_X)
    #print(np.mean(input_X),np.std(input_X))
    predictions= rfc.predict_proba(input_X)[0,1]
    predictions = round(predictions*100,0)
    return 'Attrition Rate Prediction: {}%'.format(predictions)
    


if __name__ == '__main__':
    app.run_server(mode='external', height= 500, width = '100%',port=8028,dev_tools_ui=False, debug = True)

Dash app running on http://127.0.0.1:8028/
